In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import scIB

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

scanpy==1.4.3 anndata==0.6.20 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [2]:
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR) # Ignore R warning messages
import rpy2.robjects as ro
import anndata2ri

In [8]:
ro.numpy2ri.activate()

In [3]:
plt.rcParams['figure.figsize'] = 8,8

In [76]:
adata_pre = sc.read("/home/st/strobld/Masterinternship_2019/group_malte/tabula_muris/data/processed/merged_adata.h5ad")

In [77]:
adata_p = "../scanorama.h5ad"
adata_post = sc.read(adata_p, cache=True)

In [31]:
ro.r("library(kBET)")

'kBET','tools','stats',...,'datasets','methods','base'


In [78]:
adata_pre = adata_pre[:500]

In [79]:
adata_post = adata_post[:500]

In [74]:
adata_pre

View of AnnData object with n_obs × n_vars = 100 × 12914 
    obs: 'G2M_score', 'S_score', 'barcode', 'batch', 'cell', 'cell_ontology_class', 'log_counts', 'method', 'mt_frac', 'n_counts', 'n_genes', 'phase', 'sID', 'sample', 'size_factors', 'tissue', 'louvain'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'gene_id'
    uns: 'batch_sizes', 'cell_ontology_class_colors', 'louvain_colors', 'method_colors', 'neighbors', 'paga', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    layers: 'counts'

## PC Regression

In [38]:
anndata2ri.activate()

In [6]:
#pca_data = ro.r("pca.data <- prcomp(data_mtrx, center=TRUE)")
#pca_data = adata.obsm.X_pca

In [39]:
ro.globalenv['data_mtrx'] = adata.X

ro.globalenv['pca.data'] = ro.r("pca.data <- prcomp(data_mtrx, center=TRUE)")

In [22]:
hvg = adata[:,pd.DataFrame.from_records(sc.pp.highly_variable_genes(adata, n_top_genes=20, inplace=False)).highly_variable]

In [28]:
test = hvg[:,0].X

In [29]:
ro.globalenv['y'] = test

In [46]:
hvg.shape[1]

20

In [83]:
def pcr_hvg(pre, post, n_hvg, batch):
    cons = []
    for x in pre.obs['batch'].unique():
        tmp = pre[pre.obs['batch']==x]
        tmp_post = post[post.obs['batch']==x]
        
        ro.globalenv['tmp'] = tmp.X
        ro.globalenv['tmp_post'] = tmp_post.X

        ro.globalenv['pca.data'] = ro.r("pca.data <- prcomp(tmp, center=TRUE)")
        ro.globalenv['pca_post.data'] = ro.r("pca_post.data <- prcomp(tmp_post, center=TRUE)")
        
        hvg = pd.DataFrame.from_records(sc.pp.highly_variable_genes(tmp, n_top_genes=n_hvg, inplace=False)).highly_variable
        hvg_1 = tmp[:,hvg]
        hvg_2 = tmp_post[:,hvg]
        pcr_pre_all = []
        pcr_post_all = []
        summ = 0
        for i in range(hvg_1.shape[1]):
            ro.globalenv['y_1']= hvg_1[:,i].X
            ro.globalenv['y_2']= hvg_2[:,i].X
            pcr_pre = ro.r("batch.pca <- pcRegression(pca.data, y_1, n_top=50)")
            pcr_post = ro.r("batch.pca <- pcRegression(pca_post.data, y_2, n_top=50)")
            pcr_preV = dict(zip(pcr_pre.names, list(pcr_pre)))
            pcr_postV = dict(zip(pcr_post.names, list(pcr_post)))
            pcr_pre_all.append(pcr_preV['pcRegscale'])
            pcr_post_all.append(pcr_postV['pcRegscale'])
            print(i)
        for i in range(len(pcr_pre_all)):
            diff = (pcr_pre_all[i]-pcr_post_all[i])**2
            summ = summ + diff
        cons.append(summ)
    return np.mean(cons)

In [84]:
pcr_hvg(adata_pre, adata_post, 10, 'method')

0
1
2
3
4
5
6
7
8
9


0.14029181771797983

In [51]:
pcregscale = []
for i in range(hvg.shape[1]):
    ro.globalenv['y']= hvg[:,i].X
    pcr = ro.r("batch.pca <- pcRegression(pca.data, y, n_top=50)")
    pca_regression = dict(zip(pcr.names, list(pcr)))
    pcregscale.append(pca_regression['pcRegscale'])
    print(pca_regression['pcRegscale'])

[0.55645161]
[0.71774194]
[0.75322581]
[0.64516129]
[0.73225806]
[0.65645161]
[0.72258065]
[0.72258065]
[0.27903226]
[0.66935484]
[0.87096774]
[0.80806452]
[0.83709677]
[0.65322581]
[0.88064516]
[0.62741935]
[0.32258065]
[0.80645161]
[0.44677419]
[0.81612903]


In [47]:
# ro.r("batch.silhouette <- batch_sil(pca.data, batch)")
pcr = ro.r("batch.pca <- pcRegression(pca.data, y, n_top=50)")

In [48]:
pca_regression = dict(zip(pcr.names, list(pcr)))
pca_regression

{'maxVar': array([0.8, 0.5, 0.4, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2]),
 'PmaxVar': array([1.e-16, 1.e-16, 1.e-16, 1.e-16, 1.e-16, 1.e-16, 1.e-16, 1.e-16,
        1.e-16, 1.e-16, 1.e-16, 1.e-16]),
 'pcNfrac': array([0.64]),
 'pcRegscale': array([0.55645161]),
 'maxCorr': array([0.43302997]),
 'maxR2': array([0.18751495]),
 'msigPC': array([0.98]),
 'maxsigPC': array([0.66]),
 'R2Var': array([0.00227021]),
 'ExplainedVar': array([17.3, 11.1,  7. , ...,  0. ,  0. ,  0. ]),
 'r2': array([[3.28967183e-05, 5.32569125e-01, 5.32569125e-01],
        [7.94780037e-04, 2.15389036e-03, 2.15389036e-03],
        [1.67885196e-04, 1.58566528e-01, 1.58566528e-01],
        ...,
        [6.29496017e-11, 9.99311184e-01, 9.99311184e-01],
        [9.99416180e-09, 9.91320966e-01, 9.91320966e-01],
        [1.19416610e-08, 9.90513000e-01, 9.90513000e-01]])}

In [72]:
pca_regression['pcRegscale']

array([0.65686275])